In [87]:
#pip install dash_bootstrap_components

In [2]:
#pip install selenium

In [1]:
import requests
from bs4 import BeautifulSoup 
import random
import pandas as pd
from datetime import datetime
import time
import base64

#importing visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc

In [3]:
#importing necessary packages for selenium as attendances are 
#populated using javascript therefore regular scraping will not suffice
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Running for one web page on leagueofireland.ie

In [8]:
#opening the webdriver
driver = webdriver.Chrome()  

In [9]:
#url = "https://www.leagueofireland.ie/game_centre/2283/"
url = "https://www.leagueofireland.ie/game_centre/329/"

In [10]:
driver.get(url)

In [11]:
# Handle alert if it appears
try:
    alert = WebDriverWait(driver, 2).until(EC.alert_is_present())  # Wait for alert
    alert.accept()  # Accept the alert (you can use alert.dismiss() to close it)
except:
    pass  # If no alert, just continue

try:
    # Wait for the attendance element to be populated
    game_centre_info_element = WebDriverWait(driver, 5).until(
                               EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--info"))
                                )
    score_element = WebDriverWait(driver, 5).until(
                           EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--score"))
                            )
    kick_off_element = WebDriverWait(driver, 5).until(
                           EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--kickoff"))
                            )
    
    #getting the score and game centre info
    game_score = score_element.text.strip()
    game_centre_info = game_centre_info_element.text.strip()
    kick_off_time = kick_off_element.text.strip()
    
    if game_score == 'v':
        home_goals = 'postponed'
        away_goals = 'postponed'
    else:
        home_goals_element = WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CLASS_NAME, "home-goals"))
                                )
        away_goals_element = WebDriverWait(driver, 5).until(
                               EC.presence_of_element_located((By.CLASS_NAME, "away-goals"))
                                )
        home_goals = home_goals_element.text.strip()
        away_goals = away_goals_element.text.strip()
    
except Exception as e:
    print("Error:", e)
    
finally:
    driver.quit()

In [13]:
response = requests.get(url)

if response.status_code == 200:
    page_content = response.text
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
soup = BeautifulSoup(page_content, 'html.parser')

#away_team = soup.find('h6', class_='text-center text-lg-start mb-2').get_text(strip=True)
teams = soup.find_all('span', class_='d-none d-lg-block')
# Extracting the text from each span
team_names = [team.get_text(strip=True) for team in teams]
home_team = team_names[0]
away_team = team_names[1]
    
print(f"{home_team} vs {away_team}")

Cork City FC vs Shelbourne FC


### I want to only focus on premier division games for now

In [14]:
url = "https://www.leagueofireland.ie/game_centre/4/"

In [16]:
response = requests.get(url)

if response.status_code == 200:
    page_content = response.text
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
soup = BeautifulSoup(page_content, 'html.parser')

# Find the meta tag with name="description"
meta_tag = soup.find("meta", attrs={"name": "description"})
# Extract the content
meta_content = meta_tag["content"] if meta_tag else ""

# Find the part that starts with "Premier"
for word in meta_content.split(","):
    if "Premier" in word:
        print('premier')

premier


In [17]:
title_text = soup.title.text
title_text

"League of Ireland | St Pat's v Derry"

In [18]:
# not all game centre infos are completely populated so need to account for this to avoid errors
game_centre_info

'Sat 4 Nov 2023\nMayfield United FC Astro'

In [19]:
lines = game_centre_info.split("\n")
lines

['Sat 4 Nov 2023', 'Mayfield United FC Astro']

-------
### writing a function to iterate through all the 'game centres' on leagueofireland.ie

In [20]:
def check_premier(url):
    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.text
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

    soup = BeautifulSoup(page_content, 'html.parser')

    # Find the meta tag with name="description"
    meta_tag = soup.find("meta", attrs={"name": "description"})
    # Extract the content
    meta_content = meta_tag["content"] if meta_tag else ""

    # Find the part that starts with "Premier"
    cont = False
    for word in meta_content.split(","):
        if "Premier" in word:
            cont = True
    return cont

In [21]:
check_premier("https://www.leagueofireland.ie/game_centre/329/")

False

In [22]:
def scrape_loi_webpage(url):
    #opening the webdriver
    print(f'Running for {url}')
    driver = webdriver.Chrome() 
    driver.get(url)
    
    time.sleep(.5)
    # Handle alert if it appears
    try:
        alert = WebDriverWait(driver, 2).until(EC.alert_is_present())  # Wait for alert
        alert.accept()  # Accept the alert (you can use alert.dismiss() to close it)
    except:
        pass  # If no alert, just continue

    try:
        # Wait for the attendance element to be populated
        game_centre_info_element = WebDriverWait(driver, 5).until(
                                   EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--info"))
                                    )
        score_element = WebDriverWait(driver, 5).until(
                               EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--score"))
                                )
        kick_off_element = WebDriverWait(driver, 5).until(
                               EC.presence_of_element_located((By.CLASS_NAME, "game-centre__header--kickoff"))
                                )

        #getting the score and game centre info
        game_score = score_element.text.strip()
        game_centre_info = game_centre_info_element.text.strip()
        kick_off_time = kick_off_element.text.strip()

        if game_score == 'v':
            home_goals = 'postponed'
            away_goals = 'postponed'
        else:
            home_goals_element = WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, "home-goals"))
                                    )
            away_goals_element = WebDriverWait(driver, 5).until(
                                   EC.presence_of_element_located((By.CLASS_NAME, "away-goals"))
                                    )
            home_goals = home_goals_element.text.strip()
            away_goals = away_goals_element.text.strip()

    except Exception as e:
        print("Error:", e)

    finally:
        driver.quit()
    #------------------------------#
            
    #using regular html request to get teams
    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.text
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

    soup = BeautifulSoup(page_content, 'html.parser')

    teams = soup.find_all('span', class_='d-none d-lg-block')
    # Extracting the text from each span
    team_names = [team.get_text(strip=True) for team in teams]
    home_team = team_names[0]
    away_team = team_names[1]
    
    game_centre_info = game_centre_info.split("\n")
    
    #creating dataframe
    return home_team, away_team, game_score, kick_off_time, home_goals, away_goals, game_centre_info

In [23]:
url = "https://www.leagueofireland.ie/game_centre/4454/"
data = scrape_loi_webpage(url)

Running for https://www.leagueofireland.ie/game_centre/4454/


In [24]:
data

('Drogheda United',
 'Sligo Rovers',
 '3 - 0',
 'Half Time: 1-0\nKO Time: 19:45',
 "J. Thomas 41'\nD. Markey 74'\nL. Heeney 94'",
 '',
 ['Fri 21 Feb 2025', 'David Dunne', 'Sullivan & Lambe Park', 'Att: 2,212'])

In [26]:
df = pd.DataFrame([data], columns=["home_team", "away_team", "score","kick_off_time"
                                   ,"home_goals","away_goals", "game_centre_info"])
df

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info
0,Drogheda United,Sligo Rovers,3 - 0,Half Time: 1-0\nKO Time: 19:45,J. Thomas 41'\nD. Markey 74'\nL. Heeney 94',,"[Fri 21 Feb 2025, David Dunne, Sullivan & Lamb..."


#### Looping through all game centres available on leagueofireland.ie

In [27]:
columns = ["home_team", "away_team", "score","kick_off_time","home_goals","away_goals", "game_centre_info"]
loi_df = pd.DataFrame(columns=columns)
loi_df

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info


In [30]:
for i in range (1, 4456):
    url = f"https://www.leagueofireland.ie/game_centre/{i}/"
    #run function to determine whether or not premier division game
    is_prem = check_premier(url)
    #if it is premier division game, continue with processing
    if is_prem: 
        data = scrape_loi_webpage(url)
        df = pd.DataFrame([data], columns=columns)
        loi_df =  pd.concat([loi_df, df], ignore_index=True)

In [31]:
#loi_df[loi_df['home_team'].str.contains("Sli")]

In [32]:
#loi_df.to_csv('loi_df.csv')

In [40]:
loi_df = pd.read_csv('loi_df.csv', index_col=0)
loi_df.head()

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info
0,Cork City,Bohemians,1 - 2,Half Time: 0-1\nKO Time: 19:45,R. Keating 82',19' G. Horton\n53' J. Flores,"['Fri 17 Feb 2023', 'Robert Hennessy', ""Turner..."
1,Dundalk,UCD,1 - 1,Half Time: 1-0\nKO Time: 19:45,J. Martin 34',78' C. Behan,"['Fri 17 Feb 2023', 'Arnold Hunter', 'Oriel Pa..."
2,Shelbourne,Drogheda United,0 - 0,Half Time: 0-0\nKO Time: 19:45,NaN,NaN,"['Fri 17 Feb 2023', 'Paul McLaughlin', 'Tolka ..."
3,St Patrick's Athletic,Derry City,NaN,NaN,NaN,NaN,['']
4,Sligo Rovers,Shamrock Rovers,1 - 1,Half Time: 0-1\nKO Time: 19:45,L. Browning-Lagerfeldt 94',17' G. Burke,"['Sat 18 Feb 2023', 'Damien MacGraith', 'The S..."


### Cleaning the data

In [41]:
# Extract the time using regex
loi_df['kick_off_time'] = loi_df['kick_off_time'].str.extract(r'KO Time: (\d{2}:\d{2})')

In [42]:
loi_df.head()

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info
0,Cork City,Bohemians,1 - 2,19:45,R. Keating 82',19' G. Horton\n53' J. Flores,"['Fri 17 Feb 2023', 'Robert Hennessy', ""Turner..."
1,Dundalk,UCD,1 - 1,19:45,J. Martin 34',78' C. Behan,"['Fri 17 Feb 2023', 'Arnold Hunter', 'Oriel Pa..."
2,Shelbourne,Drogheda United,0 - 0,19:45,NaN,NaN,"['Fri 17 Feb 2023', 'Paul McLaughlin', 'Tolka ..."
3,St Patrick's Athletic,Derry City,NaN,NaN,NaN,NaN,['']
4,Sligo Rovers,Shamrock Rovers,1 - 1,19:45,L. Browning-Lagerfeldt 94',17' G. Burke,"['Sat 18 Feb 2023', 'Damien MacGraith', 'The S..."


##### Analysing null values

In [46]:
loi_df.isnull().sum()

home_team             0
away_team             0
score                 1
kick_off_time         1
home_goals          103
away_goals          122
game_centre_info      0
dtype: int64

In [50]:
#removing erroneous match for now; TODO: look at webscraping for this
loi_df[loi_df['score'].isnull()]

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info
3,St Patrick's Athletic,Derry City,NaN,NaN,NaN,NaN,['']


In [55]:
loi_df = loi_df.dropna(subset=['score']).reset_index(drop=True)

In [56]:
loi_df.head()

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info
0,Cork City,Bohemians,1 - 2,19:45,R. Keating 82',19' G. Horton\n53' J. Flores,"['Fri 17 Feb 2023', 'Robert Hennessy', ""Turner..."
1,Dundalk,UCD,1 - 1,19:45,J. Martin 34',78' C. Behan,"['Fri 17 Feb 2023', 'Arnold Hunter', 'Oriel Pa..."
2,Shelbourne,Drogheda United,0 - 0,19:45,NaN,NaN,"['Fri 17 Feb 2023', 'Paul McLaughlin', 'Tolka ..."
3,Sligo Rovers,Shamrock Rovers,1 - 1,19:45,L. Browning-Lagerfeldt 94',17' G. Burke,"['Sat 18 Feb 2023', 'Damien MacGraith', 'The S..."
4,Bohemians,Dundalk,2 - 1,19:45,J. Flores 46'\nD. McDaid 71',89' J. Yli-Kokko,"['Fri 24 Feb 2023', 'Robert Harvey', 'Dalymoun..."


#### parsing the game_centre_info data

In [57]:
# Convert match_info to string to avoid errors
loi_df['game_centre_info'] = loi_df['game_centre_info'].astype(str)

# Remove brackets and split the string by ", "
loi_df[['date', 'referee', 'stadium', 'attendance']] = (
    loi_df['game_centre_info']
    .str.strip('[]')  # Remove brackets
    .str.split(', ', expand=True)  # Split by comma and space
)


In [59]:
loi_df.isnull().sum()

home_team             0
away_team             0
score                 0
kick_off_time         0
home_goals          102
away_goals          121
game_centre_info      0
date                  0
referee               0
stadium               0
attendance            0
dtype: int64

In [61]:
loi_df.head()

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info,date,referee,stadium,attendance
0,Cork City,Bohemians,1 - 2,19:45,R. Keating 82',19' G. Horton\n53' J. Flores,"['Fri 17 Feb 2023', 'Robert Hennessy', ""Turner...",17.02.2023,'Robert Hennessy',"""Turner's Cross""","'Att: 6,487'"
1,Dundalk,UCD,1 - 1,19:45,J. Martin 34',78' C. Behan,"['Fri 17 Feb 2023', 'Arnold Hunter', 'Oriel Pa...",17.02.2023,'Arnold Hunter','Oriel Park',"'Att: 3,142'"
2,Shelbourne,Drogheda United,0 - 0,19:45,NaN,NaN,"['Fri 17 Feb 2023', 'Paul McLaughlin', 'Tolka ...",17.02.2023,'Paul McLaughlin','Tolka Park',"'Att: 3,424'"
3,Sligo Rovers,Shamrock Rovers,1 - 1,19:45,L. Browning-Lagerfeldt 94',17' G. Burke,"['Sat 18 Feb 2023', 'Damien MacGraith', 'The S...",18.02.2023,'Damien MacGraith','The Showgrounds',"'Att: 4,248'"
4,Bohemians,Dundalk,2 - 1,19:45,J. Flores 46'\nD. McDaid 71',89' J. Yli-Kokko,"['Fri 24 Feb 2023', 'Robert Harvey', 'Dalymoun...",24.02.2023,'Robert Harvey','Dalymount Park',"'Att: 4,432'"


In [63]:
#formatting the date column
loi_df['date'] = pd.to_datetime(loi_df['date'].str[4:-1].str.strip(), format='%d %b %Y').dt.strftime('%d.%m.%Y')

In [66]:
##checking for any discrepencies in the attendance figures
loi_df['attendance'].str.len().unique()

array([12, 10, 13], dtype=int64)

In [68]:
# Extract numeric attendance (removing "Att: " and commas)
loi_df['attendance_new'] = (loi_df['attendance'].str.extract(r'(\d[\d,]*)')[0]
                                               .str.replace(',', '')
                                               .fillna(0).astype(int))

In [70]:
loi_df[loi_df['attendance_new']==0]

,home_team,away_team,score,kick_off_time,home_goals,away_goals,game_centre_info,date,referee,stadium,attendance,attendance_new
160,UCD,Bohemians,1 - 2,19:45,Loading...,Loading...,"['Thu 12 Oct 2023', 'Loading...', 'UCD Bowl', ...",12.10.2023,'Loading...','UCD Bowl','Loading...',0


In [76]:
##removing this erroneous value from the dataframe; TODO analysing webscraping later to fix
loi_df = loi_df[~loi_df['referee'].str.contains('Loading')]

In [79]:
loi_df = loi_df.drop(columns=['attendance']).rename(columns={'attendance_new': 'attendance'})

In [82]:
loi_df = loi_df.drop(columns=['game_centre_info'])

In [84]:
loi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 0 to 366
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   home_team      366 non-null    object
 1   away_team      366 non-null    object
 2   score          366 non-null    object
 3   kick_off_time  366 non-null    object
 4   home_goals     264 non-null    object
 5   away_goals     245 non-null    object
 6   date           366 non-null    object
 7   referee        366 non-null    object
 8   stadium        366 non-null    object
 9   attendance     366 non-null    int32 
dtypes: int32(1), object(9)
memory usage: 30.0+ KB


In [89]:
#converting date column for proper sorting
loi_df['date'] = pd.to_datetime(loi_df['date'], format='%d.%m.%Y')